<a href="https://colab.research.google.com/github/yiqiushen/Ultrasound-image-classification/blob/main/790_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download dataset from Kaggle
! pip install -q kaggle

In [ ]:
# prompt: check if data is already downloaded
import os
from google.colab import files
# Check if the dataset file already exists
dataset_filename = "breast-ultrasound-images-dataset.zip"  # Replace with the actual filename

if not os.path.exists(dataset_filename):
    # download dataset from Kaggle if not present
    ! pip install -q kaggle
    files.upload()
    # upload json file from your Kaggle account
    ! mkdir ~/.kaggle
    ! cp kaggle.json ~/.kaggle/
    ! chmod 600 ~/.kaggle/kaggle.json
    ! kaggle datasets download -d aryashah2k/breast-ultrasound-images-dataset
else:
    print(f"Dataset file '{dataset_filename}' already exists. Skipping download.")

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/aryashah2k/breast-ultrasound-images-dataset
License(s): CC0-1.0
 95% 185M/195M [00:01<00:00, 152MB/s]
100% 195M/195M [00:01<00:00, 153MB/s]


In [ ]:
# prompt: if the folder Dataset_BUSI_with_GT exists then skip unzip file otherwise unzip file

import os

# Check if the folder exists
folder_name = "Dataset_BUSI_with_GT"
if not os.path.exists(folder_name):
    # Unzip the file if the folder doesn't exist
    !unzip breast-ultrasound-images-dataset.zip
else:
    print(f"Folder '{folder_name}' already exists. Skipping unzip.")

Archive:  breast-ultrasound-images-dataset.zip
  inflating: Dataset_BUSI_with_GT/benign/benign (1).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (1)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (10).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (10)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (100).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (100)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (100)_mask_1.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (101).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (101)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (102).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (102)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (103).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (103)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (104).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (104)_mask.png  
  inflating: Data

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss.
print_every = 100
print('using device:', device)

using device: cuda


In [ ]:
# prompt: build training testing validation set using data in the folder dataset_busi_with_gt, each data point is a photo, label is the folder name benign, malignant, normal. we only need data that does not end with _mask.png

import os
import random
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from sklearn.model_selection import train_test_split

# Define data directory
data_dir = 'Dataset_BUSI_with_GT'

# Define image extensions to consider
image_extensions = ('.jpg', '.jpeg', '.png')


# Function to load and preprocess images
def load_images_and_labels(data_dir):
    images = []
    labels = []
    class_to_idx = {}
    idx_to_class = {}
    current_idx = 0
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        if os.path.isdir(class_dir):
            class_to_idx[class_name] = current_idx
            idx_to_class[current_idx] = class_name
            current_idx += 1
            for filename in os.listdir(class_dir):
                if filename.lower().endswith(image_extensions) and not filename.lower().endswith('_mask.png'):
                    img_path = os.path.join(class_dir, filename)
                    try:
                        image = Image.open(img_path)
                        images.append((image, class_name))
                    except Exception as e:
                        print(f"Error loading image {img_path}: {e}")

    return images, class_to_idx, idx_to_class

# Load images and labels
images, class_to_idx, idx_to_class = load_images_and_labels(data_dir)

# Split data
train_val_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
train_images, val_images = train_test_split(train_val_images, test_size=0.2, random_state=42)

# Separate images and labels
train_images, train_labels = zip(*train_images)
val_images, val_labels = zip(*val_images)
test_images, test_labels = zip(*test_images)

# Map class names to indices for labels
train_labels = [class_to_idx[label] for label in train_labels]
val_labels = [class_to_idx[label] for label in val_labels]
test_labels = [class_to_idx[label] for label in test_labels]

# Print split sizes
print(f"Train set size: {len(train_images)}, Labels: {len(train_labels)}")
print(f"Validation set size: {len(val_images)}, Labels: {len(val_labels)}")
print(f"Test set size: {len(test_images)}, Labels: {len(test_labels)}")




Train set size: 510, Labels: 510
Validation set size: 128, Labels: 128
Test set size: 160, Labels: 160


# Resnet with linear probe

## Just ResNet (will not work)

In [ ]:
# prompt: classify the above dataset using imagenet

import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch

# Load a pre-trained ResNet model (you can choose other models too)
model = models.resnet18(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Define the image transformations for ImageNet
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to classify a single image
def classify_image(image_path):
    try:
        img = Image.open(image_path)
        img_t = transform(img)
        batch_t = torch.unsqueeze(img_t, 0)

        with torch.no_grad():
            out = model(batch_t)

        _, index = torch.max(out, 1)

        # Load ImageNet class labels
        with open('imagenet_classes.txt') as f:
            classes = [line.strip() for line in f.readlines()]

        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100

        # Print top 5 predictions
        _, indices = torch.sort(out, descending=True)
        for idx in indices[0][:5]:
            print(f"{classes[idx]}: {percentage[idx].item():.2f}%")

    except FileNotFoundError:
        print(f"Error: Image file not found at {image_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


# Example usage: Classify an image from the dataset
# You'll need to replace 'path/to/your/image.jpg' with the actual path to an image
# from the dataset.

# Example image path
image_path_example = os.path.join(data_dir, "benign", "benign (1).png")

# Download the imagenet class labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

classify_image(image_path_example)


# To classify multiple images, you can iterate through your image list:
# for image_path in your_image_list:
#    classify_image(image_path)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 179MB/s]


--2024-12-11 00:07:35--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2024-12-11 00:07:36 (87.2 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]

grey whale: 15.82%
great white shark: 14.50%
submarine: 13.12%
tiger shark: 12.18%
dugong: 4.18%


## Linear probe
Need transfer learning (linear probe) and pseudo RGB (to deal with gray scale)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3 channels
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Prepare datasets
train_dataset = CustomDataset(train_images, train_labels, transform)
val_dataset = CustomDataset(val_images, val_labels, transform)
test_dataset = CustomDataset(test_images, test_labels, transform)

# Prepare data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load a pretrained model
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer to match the number of classes
num_classes = len(class_to_idx)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training and validation loop
num_epochs = 10
best_val_accuracy = 0.0

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Save the best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), "best_model.pth")

# Test phase
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f"Test Accuracy: {test_accuracy:.2f}%")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 70.7MB/s]


Epoch 1/10, Train Loss: 1.0222, Train Accuracy: 56.86%, Validation Loss: 1.0523, Validation Accuracy: 53.12%


KeyboardInterrupt: 

# CNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Define a custom dataset
class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure images are grayscale
    transforms.Resize((256, 256)),  # Resize to 256x256
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize for 1 channel
])
# Create datasets
train_dataset = CustomImageDataset(train_images, train_labels, transform=transform)
val_dataset = CustomImageDataset(val_images, val_labels, transform=transform)
test_dataset = CustomImageDataset(test_images, test_labels, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define a simple CNN
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # Change input channels to 1
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, loss, and optimizer
num_classes = len(class_to_idx)
model = SimpleCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.float(), labels
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        val_accuracy, val_loss = evaluate_model(model, val_loader, criterion)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

# Evaluation function
def evaluate_model(model, data_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.float(), labels
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy, val_loss / len(data_loader)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)

# Test the model
test_accuracy, _ = evaluate_model(model, test_loader, criterion)
print(f"Test Accuracy: {test_accuracy:.2f}%")


KeyboardInterrupt: 

In [ ]:
# with image resized to 128*128
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Define a custom dataset
class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure images are grayscale
    transforms.Resize((128, 128)),  # Resize to 256x256
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize for 1 channel
])
# Create datasets
train_dataset = CustomImageDataset(train_images, train_labels, transform=transform)
val_dataset = CustomImageDataset(val_images, val_labels, transform=transform)
test_dataset = CustomImageDataset(test_images, test_labels, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define a simple CNN
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # Change input channels to 1
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 32 * 64, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, loss, and optimizer
num_classes = len(class_to_idx)
model = SimpleCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.float(), labels
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        val_accuracy, val_loss = evaluate_model(model, val_loader, criterion)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

# Evaluation function
def evaluate_model(model, data_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.float(), labels
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy, val_loss / len(data_loader)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)

# Test the model
test_accuracy, _ = evaluate_model(model, test_loader, criterion)
print(f"Test Accuracy: {test_accuracy:.2f}%")


Epoch 1/10, Loss: 24.6364, Train Accuracy: 53.14%, Val Loss: 1.0264, Val Accuracy: 54.69%
Epoch 2/10, Loss: 15.1771, Train Accuracy: 57.06%, Val Loss: 1.0084, Val Accuracy: 56.25%
Epoch 3/10, Loss: 14.2479, Train Accuracy: 60.98%, Val Loss: 0.9372, Val Accuracy: 59.38%
Epoch 4/10, Loss: 13.0226, Train Accuracy: 65.10%, Val Loss: 0.8844, Val Accuracy: 65.62%
Epoch 5/10, Loss: 11.8594, Train Accuracy: 67.65%, Val Loss: 0.8467, Val Accuracy: 67.97%
Epoch 6/10, Loss: 10.4084, Train Accuracy: 72.35%, Val Loss: 0.8748, Val Accuracy: 65.62%
Epoch 7/10, Loss: 8.1102, Train Accuracy: 77.65%, Val Loss: 0.8662, Val Accuracy: 68.75%
Epoch 8/10, Loss: 6.4508, Train Accuracy: 82.75%, Val Loss: 0.8662, Val Accuracy: 72.66%
Epoch 9/10, Loss: 4.4696, Train Accuracy: 90.98%, Val Loss: 0.9806, Val Accuracy: 64.84%
Epoch 10/10, Loss: 3.1395, Train Accuracy: 94.12%, Val Loss: 1.0610, Val Accuracy: 70.31%
Test Accuracy: 69.38%


In [ ]:
class DeeperCNN(nn.Module):
    def __init__(self, num_classes):
        super(DeeperCNN, self).__init__()

        # First convolutional block
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Second convolutional block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Third convolutional block
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = None  # Placeholder for dynamic initialization
        self.fc2 = None  # Placeholder for dynamic initialization
        self.relu = nn.ReLU()

        # Output layer
        self.num_classes = num_classes

    def forward(self, x):
        # Forward pass through convolutional layers
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool1(x)

        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.pool2(x)

        x = self.relu(self.conv5(x))
        x = self.pool3(x)

        # Dynamically initialize fully connected layers based on input size
        if self.fc1 is None:
            flattened_size = x.view(x.size(0), -1).size(1)
            self.fc1 = nn.Linear(flattened_size, 512).to(x.device)
            self.fc2 = nn.Linear(512, self.num_classes).to(x.device)

        # Flatten and pass through fully connected layers
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        return x


In [ ]:
num_classes = len(class_to_idx)
model = DeeperCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)


Epoch 1/10, Loss: 16.0796, Train Accuracy: 56.86%, Val Loss: 1.0731, Val Accuracy: 54.69%
Epoch 2/10, Loss: 15.7936, Train Accuracy: 56.86%, Val Loss: 1.0577, Val Accuracy: 54.69%
Epoch 3/10, Loss: 15.6948, Train Accuracy: 57.06%, Val Loss: 1.0152, Val Accuracy: 55.47%
Epoch 4/10, Loss: 14.8139, Train Accuracy: 60.39%, Val Loss: 0.9388, Val Accuracy: 57.81%
Epoch 5/10, Loss: 14.4491, Train Accuracy: 63.73%, Val Loss: 0.9206, Val Accuracy: 58.59%
Epoch 6/10, Loss: 14.0032, Train Accuracy: 61.96%, Val Loss: 0.9865, Val Accuracy: 58.59%
Epoch 7/10, Loss: 13.3172, Train Accuracy: 64.90%, Val Loss: 0.9788, Val Accuracy: 58.59%
Epoch 8/10, Loss: 12.7633, Train Accuracy: 65.88%, Val Loss: 0.8929, Val Accuracy: 61.72%
Epoch 9/10, Loss: 11.9959, Train Accuracy: 67.65%, Val Loss: 0.8603, Val Accuracy: 65.62%
Epoch 10/10, Loss: 11.1595, Train Accuracy: 70.20%, Val Loss: 0.8506, Val Accuracy: 64.06%


# Vision-language model

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

# Load the CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

## Zero-shot

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

# Load the CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)


In [ ]:


def zero_shot_classify(image_paths, class_labels):
    results = {}
    for img_path in image_paths:
        image = Image.open(img_path)

        # Process the image and text labels
        inputs = processor(
            text=class_labels,
            images=image,
            return_tensors='pt',
            padding=True,
        ).to(device)

        # Perform classification
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image  # Image-to-text similarity scores
        probabilities = logits_per_image.softmax(dim=1)

        # Store probabilities
        results[img_path] = {label: prob.item() for label, prob in zip(class_labels, probabilities[0])}

    return results

# Dynamically generate test image paths based on dataset structure
test_image_paths = [
    os.path.join(data_dir, idx_to_class[label], img.filename) for img, label in zip(test_images, test_labels)
]

# Define class labels (replace with your actual class labels)
class_labels = list(class_to_idx.keys()) # Get class labels from class_to_idx dictionary

# Perform zero-shot classification
zero_shot_results = zero_shot_classify(test_image_paths, class_labels)

# Print results
for img_path, probs in zero_shot_results.items():
    print(f'Results for {img_path}:')
    for label, prob in probs.items():
        print(f'  {label}: {prob:.4f}')

Results for /content/Dataset_BUSI_with_GT/malignant/malignant (27).png:
  normal: 0.1170
  benign: 0.2119
  malignant: 0.6711
Results for /content/Dataset_BUSI_with_GT/malignant/malignant (70).png:
  normal: 0.1260
  benign: 0.2084
  malignant: 0.6655
Results for /content/Dataset_BUSI_with_GT/normal/normal (107).png:
  normal: 0.0732
  benign: 0.4005
  malignant: 0.5263
Results for /content/Dataset_BUSI_with_GT/benign/benign (391).png:
  normal: 0.0809
  benign: 0.3160
  malignant: 0.6031
Results for /content/Dataset_BUSI_with_GT/normal/normal (9).png:
  normal: 0.1072
  benign: 0.1650
  malignant: 0.7278
Results for /content/Dataset_BUSI_with_GT/malignant/malignant (123).png:
  normal: 0.2040
  benign: 0.3588
  malignant: 0.4372
Results for /content/Dataset_BUSI_with_GT/benign/benign (4).png:
  normal: 0.1113
  benign: 0.2581
  malignant: 0.6306
Results for /content/Dataset_BUSI_with_GT/benign/benign (317).png:
  normal: 0.0512
  benign: 0.2028
  malignant: 0.7460
Results for /content

In [ ]:
# labels can be replaced by more descriptive langauage
class_labels

['normal', 'benign', 'malignant']

In [ ]:

# Create a mapping from numerical indices to the new descriptive labels
idx_to_new_label = {
    0: 'an ultrasonic image of normal breast tissue',
    1: 'an ultrasonic image of benign breast tumor',
    2: 'an ultrasonic image of malignant breast tumor',
}

# Convert the numerical labels in train/val/test datasets to the new descriptive labels
train_labels_new = [idx_to_new_label[label] for label in train_labels]
val_labels_new = [idx_to_new_label[label] for label in val_labels]
test_labels_new = [idx_to_new_label[label] for label in test_labels]


# Now, use these new labels when creating your datasets:

# Create datasets with new labels
train_dataset = CustomDataset(train_images, train_labels_new, transform=transform)
val_dataset = CustomDataset(val_images, val_labels_new, transform=transform)
test_dataset = CustomDataset(test_images, test_labels_new, transform=transform)




In [ ]:

class_to_idx = {label: idx for idx, label in enumerate(class_labels)}
class_to_idx


{'normal': 0, 'benign': 1, 'malignant': 2}

In [ ]:
# prompt: label the test data by the highest probability and calculate the test error

# Function to label test data and calculate test error
def label_and_evaluate(zero_shot_results, test_labels, class_to_idx):
    correct_predictions = 0
    total_predictions = len(test_labels)
    predicted_labels = []

    for img_path, probs in zero_shot_results.items():
        # Find the class label with the highest probability
        predicted_label = max(probs, key=probs.get)
        predicted_labels.append(predicted_label)

        # Convert predicted label to numerical index
        try:
            predicted_label_index = list(class_to_idx.keys()).index(predicted_label)
        except ValueError:
          print(f"Warning: Predicted label '{predicted_label}' not found in class_to_idx. Skipping.")
          continue

        # Get the true label for the current image
        # Assuming test_labels contains numerical indices corresponding to class_to_idx
        true_label_index = test_labels[list(zero_shot_results.keys()).index(img_path)]

        if predicted_label_index == true_label_index:
            correct_predictions += 1

    test_error = 1 - (correct_predictions / total_predictions)
    print(f"Test Error: {test_error:.4f}")
    return test_error


# Assuming you have test_labels and class_to_idx defined
predicted_labels = label_and_evaluate(zero_shot_results, test_labels, class_to_idx)

Test Error: 0.6375


## Fine Tuning

In [ ]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim

# Transformations for training and validation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Create datasets and data loaders
train_dataset = CustomDataset(train_images, train_labels, transform)
val_dataset = CustomDataset(val_images, val_labels, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load the CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
# Modify the CLIP model for fine-tuning
vision_model = model.vision_model  # Use only the vision encoder
num_classes = len(class_labels)

# Modify the CLIP model for fine-tuning
vision_model = model.vision_model  # Use only the vision encoder
num_classes = len(class_labels)

# Get the output size of the vision model
with torch.no_grad():
    output_size = vision_model(torch.randn(1, 3, 224, 224).to(device)).pooler_output.shape[1]

# Add a new classification head with the correct input size
classifier = nn.Linear(output_size, num_classes)
vision_model.classifier = classifier.to(device)


# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Fine-tuning loop
num_epochs = 10
vision_model.train()

for epoch in range(num_epochs):
    total_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = vision_model(images)
        outputs = vision_model.classifier(outputs.pooler_output) # Extract pooler_output
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    # Validation loop
    vision_model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = vision_model(images)
            outputs = vision_model.classifier(outputs.pooler_output)
            loss = criterion(outputs, labels)


            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    train_acc = 100 * correct / total
    val_acc = 100 * correct_val / total_val
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}, "
          f"Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

# Save the fine-tuned model
torch.save(vision_model.state_dict(), "clip_fine_tuned.pth")


Epoch 1/10, Loss: 0.9330, Train Acc: 59.41%, Val Acc: 60.16%
Epoch 2/10, Loss: 0.7797, Train Acc: 65.49%, Val Acc: 51.56%
Epoch 3/10, Loss: 0.7151, Train Acc: 70.59%, Val Acc: 66.41%
Epoch 4/10, Loss: 0.6558, Train Acc: 70.59%, Val Acc: 74.22%
Epoch 5/10, Loss: 0.6149, Train Acc: 73.92%, Val Acc: 69.53%
Epoch 6/10, Loss: 0.5781, Train Acc: 78.04%, Val Acc: 69.53%
Epoch 7/10, Loss: 0.5687, Train Acc: 76.86%, Val Acc: 73.44%
Epoch 8/10, Loss: 0.5436, Train Acc: 80.20%, Val Acc: 77.34%
Epoch 9/10, Loss: 0.5148, Train Acc: 81.57%, Val Acc: 73.44%
Epoch 10/10, Loss: 0.4990, Train Acc: 81.18%, Val Acc: 76.56%


In [ ]:
# prompt: print out the label predicted on the test set, the true label of the test set and test error based on the fine tuned model

# Load the fine-tuned model
vision_model.load_state_dict(torch.load("clip_fine_tuned.pth"))
vision_model.eval()

# Create the test dataset and data loader
test_dataset = CustomDataset(test_images, test_labels, transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Perform predictions on the test set
predicted_labels = []
true_labels = []
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = vision_model(images)
        outputs = vision_model.classifier(outputs.pooler_output)
        _, predicted = torch.max(outputs, 1)

        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate test error
test_error = 1 - (correct / total)

# Print the results
print("Predicted Labels:", predicted_labels)
print("True Labels:", true_labels)
print(f"Test Error: {test_error:.4f}")

<ipython-input-25-4bfcc33809f0>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vision_model.load_state_dict(torch.load("clip_fine_tuned.pth"))


Predicted Labels: [1, 2, 0, 1, 0, 1, 1, 0, 2, 1, 0, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 0, 1, 1, 1, 2, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 2, 1, 0, 2, 1, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 2, 1, 1, 1, 1, 0, 2, 1, 1, 1, 0, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 1, 2]
True Labels: [2, 2, 0, 1, 0, 2, 1, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 0, 1, 2, 2, 1, 1, 0, 1, 1, 0, 0, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 0, 2, 1, 0, 1, 1, 2, 0, 0, 1, 1, 1, 2, 1, 1, 1, 0, 0, 1, 2, 2, 1, 1, 0, 2, 2, 1, 1, 2, 2, 2, 2, 0, 2, 1, 1, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 0, 0, 2, 0, 1, 2, 1, 1, 1, 1, 0, 2, 1, 1, 2, 0, 1, 2, 2, 1, 0, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 1, 2]
Test Er